<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/query_engine/knowledge_graph_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>


# 知识图谱查询引擎

创建知识图谱通常涉及专业化和复杂的任务。然而，通过利用Llama Index (LLM)、KnowledgeGraphIndex和GraphStore，我们可以从[Llama Hub](https://llamahub.ai/)支持的任何数据源中促进相对有效的知识图谱的创建。

此外，查询知识图谱通常需要与存储系统相关的领域特定知识，比如Cypher。但是，借助LLM和LlamaIndex KnowledgeGraphQueryEngine的帮助，可以使用自然语言来完成这项任务！

在这个演示中，我们将指导您完成以下步骤：

- 使用Llama Index提取和设置知识图谱
- 使用Cypher查询知识图谱
- 使用自然语言查询知识图谱


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai

In [ ]:
!pip install llama-index

让我们首先为Llama指数的基本准备工作做好准备。


OpenAI 是一个人工智能研究实验室，致力于推动人工智能的发展，使其造福全人类。该实验室的使命是确保人工智能的发展对人类的利益最大化。 OpenAI 的目标是开发出一种能够广泛应用的人工智能，同时确保这种技术的安全性和可控性。


In [ ]:
# 为OpenAIimport osos.environ["OPENAI_API_KEY"] = "sk-..."import loggingimport syslogging.basicConfig(    stream=sys.stdout, level=logging.INFO)  # logging.DEBUG 用于更详细的输出# 定义LLMfrom llama_index.llms.openai import OpenAIfrom llama_index.core import SettingsSettings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")Settings.chunk_size = 512

### Azure


In [ ]:
from llama_index.llms.azure_openai import AzureOpenAIfrom llama_index.embeddings.azure_openai import AzureOpenAIEmbedding# 对于Azure OpenAIapi_key = "<api-key>"azure_endpoint = "https://<your-resource-name>.openai.azure.com/"api_version = "2023-07-01-preview"llm = AzureOpenAI(    model="gpt-35-turbo-16k",    deployment_name="my-custom-llm",    api_key=api_key,    azure_endpoint=azure_endpoint,    api_version=api_version,)# 您需要部署自己的嵌入模型以及自己的聊天完成模型embed_model = AzureOpenAIEmbedding(    model="text-embedding-ada-002",    deployment_name="my-custom-embedding",    api_key=api_key,    azure_endpoint=azure_endpoint,    api_version=api_version,)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

## 准备NebulaGraph

在进入下一步创建知识图谱之前，让我们确保我们已经运行了NebulaGraph，并定义了数据模式。


In [ ]:
# 使用以下命令创建一个 NebulaGraph（版本 3.5.0 或更新版本）集群：# 选项 0 适用于使用 Docker 的机器：`curl -fsSL nebula-up.siwei.io/install.sh | bash`# 选项 1 适用于桌面版：NebulaGraph Docker 扩展 https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext# 如果不是使用上述方法，可以使用以下命令从 NebulaGraph 控制台创建：# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);# :sleep 10;# USE llamaindex;# CREATE TAG entity(name string);# CREATE EDGE relationship(relationship string);# :sleep 10;# CREATE TAG INDEX entity_index ON entity(name(256));%pip install ipython-ngql nebula3-pythonos.environ["NEBULA_USER"] = "root"os.environ["NEBULA_PASSWORD"] = "nebula"  # 默认为 "nebula"os.environ[    "NEBULA_ADDRESS"] = "127.0.0.1:9669"  # 假设已在本地安装了 NebulaGraphspace_name = "llamaindex"edge_types, rel_prop_names = ["relationship"], [    "relationship"]  # 默认值，如果从空的知识图谱中创建，则可以省略tags = ["entity"]  # 默认值，如果从空的知识图谱中创建，则可以省略

You should consider upgrading via the '/Users/loganmarkewich/llama_index/llama-index/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


准备好使用graph_store作为NebulaGraphStore的StorageContext


In [ ]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## （可选）使用LlamaIndex构建知识图谱

在Llama Index和LLM的帮助下，我们可以从给定的文档构建知识图谱。

如果我们已经在NebulaGraphStore上有一个知识图谱，那么这一步可以跳过。


### 第一步，从维基百科加载“银河护卫队3”电影的数据


In [ ]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

### 第2步，使用NebulaGraph作为图存储生成知识图索引

接下来，我们将创建一个KnowledgeGraphIndex来启用基于图的RAG，请参阅[此处](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html)了解详情，除此之外，我们还可以为其他目的运行一个知识图。


In [ ]:
from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

现在我们在名为`llamaindex`的NebulaGraph集群上有一个有关《银河护卫队3》电影的知识图谱，让我们稍微玩一下。


In [ ]:
# 安装相关的包，密码默认为 nebula%pip install ipython-ngql networkx pyvis%load_ext ngql%ngql --address 127.0.0.1 --port 9669 --user root --password <password>

You should consider upgrading via the '/Users/loganmarkewich/llama_index/llama-index/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
[ERROR]:
 'IPythonNGQL' object has no attribute '_decode_value'


Name
0  llamaindex

In [ ]:
# 使用Cypher查询一些随机的关系%ngql 使用 llamaindex;%ngql 匹配 ()-[e]->() 返回 e 限制 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


e
0  ("A second trailer for the film")-[:relationsh...
1  ("Adam McKay")-[:relationship@-442854342936029...
2  ("Adam McKay")-[:relationship@8513344855738553...
3  ("Asim Chaudhry")-[:relationship@-803614038978...
4  ("Bakalova")-[:relationship@-25325064520311626...
5  ("Bautista")-[:relationship@-90386029986457371...
6  ("Bautista")-[:relationship@-90386029986457371...
7  ("Beth Mickle")-[:relationship@716197657641767...
8  ("Bradley Cooper")-[:relationship@138630731832...
9  ("Bradley Cooper")-[:relationship@838402633192...

In [ ]:
# 绘制结果%ng_draw

nebulagraph_draw.html


## 查询知识图谱

最后，让我们演示如何使用自然语言查询知识图谱！

在这里，我们将利用`KnowledgeGraphQueryEngine`，并将`NebulaGraphStore`作为`storage_context.graph_store`。


In [ ]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    llm=llm,
    verbose=True,
)

In [ ]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
```
MATCH (p:`entity`)-[:relationship]->(m:`entity`) WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`;
```
Graph Store Response:
{'p.entity.name': ['Peter Quill', 'Peter Quill', 'Peter Quill', 'Peter Quill', 'Peter Quill']}
Final Response: 

Peter Quill is a character in the Marvel Universe. He is the son of Meredith Quill and Ego the Living Planet.


<b>

Peter Quill is a character in the Marvel Universe. He is the son of Meredith Quill and Ego the Living Planet.</b>

In [ ]:
graph_query = query_engine.generate_query(    "告诉我关于彼得·奎尔的信息?",)graph_query = graph_query.replace("WHERE", "\n  WHERE").replace(    "RETURN", "\nRETURN")display(    Markdown(        f"""{graph_query}"""    ))


```cypher
```
MATCH (p:`entity`)-[:relationship]->(m:`entity`) 
  WHERE p.`entity`.`name` == 'Peter Quill'

RETURN p.`entity`.`name`;
```
```


我们可以看到它有助于生成图查询：

```cypher
MATCH (p:`entity`)-[:relationship]->(e:`entity`) 
  WHERE p.`entity`.`name` == 'Peter Quill' 
RETURN e.`entity`.`name`;
```
并根据其结果综合出问题：

```json
{'e2.entity.name': ['祖父', 'Gamora的另一个版本', '银河护卫队']}
```


当然，我们仍然可以查询它！而且这个查询引擎可以成为我们最好的图查询语言学习机器人。 :)


In [ ]:
%%ngql 
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


p.entity.name           e.relationship  \
0   Peter Quill  would return to the MCU   
1   Peter Quill  was abducted from Earth   
2   Peter Quill             is leader of   
3   Peter Quill            was raised by   
4   Peter Quill            is half-human   

                            m.entity.name  
0                                May 2021  
1                              as a child  
2                 Guardians of the Galaxy  
3  a group of alien thieves and smugglers  
4                          half-Celestial

并且将查询更改为被渲染


In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


p  \
0  ("Peter Quill" :entity{name: "Peter Quill"})   
1  ("Peter Quill" :entity{name: "Peter Quill"})   
2  ("Peter Quill" :entity{name: "Peter Quill"})   
3  ("Peter Quill" :entity{name: "Peter Quill"})   
4  ("Peter Quill" :entity{name: "Peter Quill"})   

                                                   e  \
0  ("Peter Quill")-[:relationship@-84437522554765...   
1  ("Peter Quill")-[:relationship@-11770408155938...   
2  ("Peter Quill")-[:relationship@-79394488349732...   
3  ("Peter Quill")-[:relationship@325695233021653...   
4  ("Peter Quill")-[:relationship@555553046209276...   

                                                   m  
0             ("May 2021" :entity{name: "May 2021"})  
1         ("as a child" :entity{name: "as a child"})  
2  ("Guardians of the Galaxy" :entity{name: "Guar...  
3  ("a group of alien thieves and smugglers" :ent...  
4  ("half-Celestial" :entity{name: "half-Celestia...

In [ ]:
%ng_draw

nebulagraph_draw.html


这个知识获取查询的结果在渲染的图表中表现得再清晰不过了。
